# Destiny 2 Weapon Roll Extraction
This notebook interactively extracts and classifies all possible and equipped Destiny 2 weapon perks for your account, similar to D2Checklist or DIM.

In [1]:
# --- Imports and Environment Setup ---
import os
import sys
import json
from pprint import pprint
from datetime import datetime
from dotenv import load_dotenv
from supabase import create_client

# Adjust sys.path to include the project root
SCRIPT_DIR = os.path.dirname(os.path.realpath("."))
PROJECT_ROOT = os.path.abspath(os.path.join(SCRIPT_DIR, os.pardir))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# Load environment variables
load_dotenv()
BUNGIE_API_KEY = os.getenv("BUNGIE_API_KEY")
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_SERVICE_ROLE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")
BUNGIE_ACCESS_TOKEN = os.getenv("BUNGIE_ACCESS_TOKEN")

# Logging
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


In [2]:
import os
import sys

# Set this to the absolute path of your project root (where web_app/ lives)
PROJECT_ROOT = "/Users/davehiltbrand/Documents/destiny2_catalysts"
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
print("sys.path:", sys.path)

sys.path: ['/Users/davehiltbrand/Documents/destiny2_catalysts', '/Users/davehiltbrand/Documents', '/opt/homebrew/Cellar/python@3.11/3.11.12/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.12/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.12/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/davehiltbrand/Documents/destiny2_catalysts/venv/lib/python3.11/site-packages']


In [3]:
TOKEN_PATH = os.path.join(PROJECT_ROOT, "token.json")

with open(TOKEN_PATH, "r") as f:
    token_data = json.load(f)
print(token_data)

{'access_token': 'CIilBxKGAgAgwl7sm+Nrb+Z6BLZwm/c1Hb/p9uKuXYyYWz9JMKIdY4ngAAAAOGxeRPQMRY8Ep32CyQIJ41Y7SIQyPVf/VLUTJBrWbdUPIG+piUeA/+hyFyLaJVeIPy+uFEsONQijbTxDr2pSxbWtqbVAXIqI80lL3Be0LLsfDSbA3DEbCUfMArySep6+AosIOUdq5swZxh/uS0lQmx7NWFhkon32XHoMhrc8TZE2FfTlrs5TVAmMPh6mYbkKuUvI52Y+DmqrgYEZNDHM64SL0vzcUPzKuII/4LzhlNr9KaxmSjCe5gM8RpvK/6uRCvErVAc/cELKsa8nnyUtYfWXaQUwisF+ZJE3uUVX3mE=', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'CIilBxKGAgAgECulzix6v/kR+pFcA9koH2KUE9kSqFVbvqpglr3oOfPgAAAARDbs96N/zyFjkO2b0VZqU8MMS6pvQBxE70ROu8mIQCFOG7AY1dID4RKjzp1kEjAf/ydtjuMCJTAh0DhiypT1AQIkbxO90I2okWl5Vt5R6Cuir6yRh0RGpt4s/x7xmnEu7z6P7tOr+FqWTF7wAVFx4l8AYjskhAGnU//Ssp7SQWiOKRXTC1oEZA7ilBHISC4xKsrokMvWLgnjQqRrDg447S6kv0Z7VfHHblBsOKMeqbyfPR6mcp0bqmTnozfAeV4g3g7M/VcO6B78iACZlVYq4+LDmHFA8WqGUcG7BrT7Lcg=', 'refresh_expires_in': 7776000, 'membership_id': '29565467', 'received_at': '2025-05-14T06:49:44.548056'}


In [4]:
import os

# Set this to your project root where token.json lives
PROJECT_ROOT = "/Users/davehiltbrand/Documents/destiny2_catalysts"
os.chdir(PROJECT_ROOT)
print("Current working directory:", os.getcwd())
print("token.json exists:", os.path.exists("token.json"))

Current working directory: /Users/davehiltbrand/Documents/destiny2_catalysts
token.json exists: True


In [5]:
# --- Supabase and Bungie API Setup ---
from web_app.backend.weapon_api import WeaponAPI
from web_app.backend.manifest import SupabaseManifestService
from web_app.backend.bungie_oauth import OAuthManager
# Add the parent directory to sys.path to find web_app module


sb_client = create_client(SUPABASE_URL, SUPABASE_SERVICE_ROLE_KEY)
manifest_service = SupabaseManifestService(sb_client=sb_client)
oauth_manager = OAuthManager()
weapon_api = WeaponAPI(oauth_manager=oauth_manager, manifest_service=manifest_service)


2025-05-14 06:51:14,731 - INFO - [DEBUG] Attempting to load token data from file...
2025-05-14 06:51:14,732 - INFO - [DEBUG] Raw data loaded from token.json: {'access_token': 'CIilBxKGAgAgwl7sm+Nrb+Z6BLZwm/c1Hb/p9uKuXYyYWz9JMKIdY4ngAAAAOGxeRPQMRY8Ep32CyQIJ41Y7SIQyPVf/VLUTJBrWbdUPIG+piUeA/+hyFyLaJVeIPy+uFEsONQijbTxDr2pSxbWtqbVAXIqI80lL3Be0LLsfDSbA3DEbCUfMArySep6+AosIOUdq5swZxh/uS0lQmx7NWFhkon32XHoMhrc8TZE2FfTlrs5TVAmMPh6mYbkKuUvI52Y+DmqrgYEZNDHM64SL0vzcUPzKuII/4LzhlNr9KaxmSjCe5gM8RpvK/6uRCvErVAc/cELKsa8nnyUtYfWXaQUwisF+ZJE3uUVX3mE=', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'CIilBxKGAgAgECulzix6v/kR+pFcA9koH2KUE9kSqFVbvqpglr3oOfPgAAAARDbs96N/zyFjkO2b0VZqU8MMS6pvQBxE70ROu8mIQCFOG7AY1dID4RKjzp1kEjAf/ydtjuMCJTAh0DhiypT1AQIkbxO90I2okWl5Vt5R6Cuir6yRh0RGpt4s/x7xmnEu7z6P7tOr+FqWTF7wAVFx4l8AYjskhAGnU//Ssp7SQWiOKRXTC1oEZA7ilBHISC4xKsrokMvWLgnjQqRrDg447S6kv0Z7VfHHblBsOKMeqbyfPR6mcp0bqmTnozfAeV4g3g7M/VcO6B78iACZlVYq4+LDmHFA8WqGUcG7BrT7Lcg=', 'refresh_expires_in': 7776000, 'memb

In [21]:
# --- Fetch Profile and Prepare Data ---
membership_info = await weapon_api.get_membership_info()
if not membership_info:
    raise Exception("Could not fetch membership info from Bungie API.")
membership_type = membership_info["type"]
destiny_membership_id = membership_info["id"]

components = [102, 201, 205, 305, 310]
profile_response_data = await weapon_api.get_profile(
    membership_type=membership_type,
    destiny_membership_id=destiny_membership_id,
    components=components
)

if not profile_response_data:
    raise Exception("Failed to get profile response data from WeaponAPI.")
profile_data = profile_response_data["Response"]
character_equip = profile_data["characterEquipment"]["data"]
character_inventories = profile_data["characterInventories"]["data"]
profile_inventory = profile_data["profileInventory"]["data"]
item_sockets = profile_data["itemComponents"]["sockets"]["data"]
# item_instances = profile_data["itemComponents"]["instances"]["data"]
# profile_plugsets = profile_data.get("profilePlugSets", {}).get("data", {}).get("plugs", {})
# character_plugsets = profile_data.get("characterPlugSets", {}).get("data", {})
reusable_plugs = profile_data['itemComponents']['reusablePlugs']['data']


2025-05-14 07:07:48,526 - INFO - WeaponAPI: Fetching membership info...
2025-05-14 07:07:48,652 - INFO - WeaponAPI found membership - Type: 3, ID: 4611686018517808552
2025-05-14 07:07:48,653 - INFO - WeaponAPI: Getting profile for 4611686018517808552, type 3 with components [102, 201, 205, 305, 310]
2025-05-14 07:07:48,926 - INFO - WeaponAPI successfully fetched profile components for user 4611686018517808552.


In [20]:
profile_data['itemComponents'].keys()

dict_keys(['sockets', 'reusablePlugs'])

In [22]:
# --- Flatten all items and process up to 10 weapons ---
from itertools import chain
all_items = list(chain(
    *(v.get("items", []) for v in character_equip.values()),
    *(v.get("items", []) for v in character_inventories.values()),
    profile_inventory.get("items", []),
))



In [25]:
import random
random.shuffle(all_items)
for item in all_items:
    item_hash = item.get('itemHash')
    item_def = manifest_service.get_definition('DestinyInventoryItemDefinition', item_hash)
    if item_def.get('itemType') == 3:  # not a weapon
        break

instance_id = item.get('itemInstanceId')
print(f'Processing {item_def["displayProperties"]["name"]} ({instance_id})')

2025-05-14 07:09:07,457 - INFO - HTTP Request: GET https://grwqemflswabswphkute.supabase.co/rest/v1/destinyinventoryitemdefinition?select=json_data&hash=eq.2549387168 "HTTP/2 200 OK"
2025-05-14 07:09:07,656 - INFO - HTTP Request: GET https://grwqemflswabswphkute.supabase.co/rest/v1/destinyinventoryitemdefinition?select=json_data&hash=eq.1201830623 "HTTP/2 200 OK"


Processing Truth (6917529600276329350)


In [30]:
# fighting lion 6917529770813290935
# Find Fighting Lion in all items
fighting_lion_hash = 6917529770813290935 # Fighting Lion's item hash
fighting_lion = None
for item in all_items:
    if item.get('itemHash') == fighting_lion_hash:
        fighting_lion = item
        break

if fighting_lion:
    instance_id = fighting_lion.get('itemInstanceId')
    item_def = manifest_service.get_definition('DestinyInventoryItemDefinition', fighting_lion_hash)
    print(f'Found Fighting Lion ({instance_id})')
else:
    print('Fighting Lion not found in inventory')


Fighting Lion not found in inventory


In [31]:
item_sockets.get(instance_id, {})

{'sockets': [{'plugHash': 2491817779, 'isEnabled': True, 'isVisible': True},
  {'plugHash': 1478423395, 'isEnabled': True, 'isVisible': True},
  {'plugHash': 2822142346, 'isEnabled': True, 'isVisible': True},
  {'plugHash': 2911329003, 'isEnabled': True, 'isVisible': True},
  {'plugHash': 3465198467, 'isEnabled': True, 'isVisible': True},
  {'plugHash': 2931483505, 'isEnabled': True, 'isVisible': True},
  {'isEnabled': False, 'isVisible': False},
  {'isEnabled': False, 'isVisible': False},
  {'isEnabled': False, 'isVisible': False},
  {'plugHash': 2240097604, 'isEnabled': True, 'isVisible': True},
  {'plugHash': 1498917124, 'isEnabled': True, 'isVisible': False}]}

In [32]:
reusable_plugs_data = profile_response_data.get("Response", {}).get("itemComponents", {}).get("reusablePlugs", {}).get("data", {})
print(f"[DEBUG] reusable_plugs_data keys: {list(reusable_plugs_data.keys())} (count: {len(reusable_plugs_data)})")
instance_reusable_plugs = reusable_plugs_data.get(instance_id, {}).get('plugs', {})
instance_reusable_plugs

[DEBUG] reusable_plugs_data keys: ['6917530110443598914', '6917529603870481739', '6917529871647767419', '6917529819069510174', '6917529617647242171', '6917529662540121834', '6917529801112623330', '6917529806490869337', '6917529835255814812', '6917529990397295727', '6917529867795149587', '6917529475949466141', '6917529619688663567', '6917529934675429376', '6917529934675429899', '6917529933980104631', '6917529930190442138', '6917529931288722132', '6917529931589518556', '6917529937560018316', '6917529935705442395', '6917529607550277784', '6917529686490795757', '6917529932278441412', '6917529930120527063', '6917529930113795729', '6917529930543808058', '6917529930106867061', '6917529930120529872', '6917529878388215364', '6917529990397296072', '6917529797961593979', '6917529607550275197', '6917530111462386496', '6917529941047491787', '6917530110796452069', '6917530111444227557', '6917530111464930061', '6917530110803967924', '6917530111464931450', '6917530110553875057', '6917530111460872952',

{}

In [33]:
instance_sockets = item_sockets.get(instance_id, {}).get('sockets', {})
instance_sockets

[{'plugHash': 2491817779, 'isEnabled': True, 'isVisible': True},
 {'plugHash': 1478423395, 'isEnabled': True, 'isVisible': True},
 {'plugHash': 2822142346, 'isEnabled': True, 'isVisible': True},
 {'plugHash': 2911329003, 'isEnabled': True, 'isVisible': True},
 {'plugHash': 3465198467, 'isEnabled': True, 'isVisible': True},
 {'plugHash': 2931483505, 'isEnabled': True, 'isVisible': True},
 {'isEnabled': False, 'isVisible': False},
 {'isEnabled': False, 'isVisible': False},
 {'isEnabled': False, 'isVisible': False},
 {'plugHash': 2240097604, 'isEnabled': True, 'isVisible': True},
 {'plugHash': 1498917124, 'isEnabled': True, 'isVisible': False}]

In [39]:
socket_plug_hashes = {}
instance_reusable_plugs = reusable_plugs_data.get(instance_id, {}).get('plugs', {})
instance_sockets = item_sockets.get(instance_id, {}).get('sockets', [])

for idx, socket in enumerate(instance_sockets):
    # Prefer reusablePlugs if present for this socket
    plug_hashes = []
    if instance_reusable_plugs and str(idx) in instance_reusable_plugs:
        plug_hashes = [plug['plugItemHash'] for plug in instance_reusable_plugs[str(idx)]]
    # Always add the equipped plug from itemSockets (especially for intrinsic)
    plug_hash = socket.get('plugHash')
    if plug_hash and plug_hash not in plug_hashes:
        plug_hashes.append(plug_hash)
    if plug_hashes:
        socket_plug_hashes[idx] = plug_hashes

In [40]:
all_plug_hashes = set()
for plug_list in socket_plug_hashes.values():
    all_plug_hashes.update(plug_list)

In [41]:
# 2. Batch fetch all plug definitions
plug_definitions = manifest_service.get_definitions_batch(
    'DestinyInventoryItemDefinition',
    list(all_plug_hashes)
)

2025-05-14 11:28:44,397 - INFO - Starting batch fetch for 8 definitions from destinyinventoryitemdefinition in 1 chunk(s).


2025-05-14 11:28:44,853 - INFO - HTTP Request: GET https://grwqemflswabswphkute.supabase.co/rest/v1/destinyinventoryitemdefinition?select=hash%2Cjson_data&hash=in.%281478423395%2C3465198467%2C2240097604%2C1498917124%2C2822142346%2C2911329003%2C2931483505%2C2491817779%29 "HTTP/2 200 OK"
2025-05-14 11:28:44,871 - INFO - Batch fetch complete for destinyinventoryitemdefinition. Total definitions fetched: 8 out of 8 requested.


In [42]:
socket_plug_defs = {}
for socket_index, plug_hashes in socket_plug_hashes.items():
    socket_plug_defs[socket_index] = [
        plug_definitions.get(plug_hash) for plug_hash in plug_hashes if plug_definitions.get(plug_hash)
    ]

In [43]:
# Assuming socket_plug_defs is already built as described earlier
for socket_index, plug_defs in socket_plug_defs.items():
    plug_names = [plug_def['displayProperties']['name'] for plug_def in plug_defs if plug_def]
    print(f"Socket {socket_index}: {plug_names}")

Socket 0: ['Prototype Trueseeker']
Socket 1: ['Volatile Launch']
Socket 2: ['High-Velocity Rounds']
Socket 3: ['Grenades and Horseshoes']
Socket 4: ['Composite Stock']
Socket 5: ['Default Ornament']
Socket 9: ['Kill Tracker']
Socket 10: ['Empty Catalyst Socket']


In [21]:
# Define your PCI/category mappings
PCI_COL1 = {"barrels", "tubes", "bowstrings", "blades", "hafts", "scopes"}
PCI_COL2 = {"magazines", "batteries", "guards", "arrows"}
TRAIT_PCI = {"frames", "grips", "traits"}
ORIGIN_PCI = {"origin"}
MASTERWORK_PCI = {"masterworks"}

def get_plug_category(plug_def):
    pci = plug_def.get('plug', {}).get('plugCategoryIdentifier', '').lower()
    name = plug_def.get('displayProperties', {}).get('name', '')
    item_type_display_name = plug_def.get('itemTypeDisplayName', '').lower()
    if any(key in pci for key in PCI_COL1):
        return "col1_barrel"
    elif any(key in pci for key in PCI_COL2):
        return "col2_magazine"
    elif any(key in pci for key in TRAIT_PCI) or plug_def.get('itemTypeDisplayName') in ("Trait", "Enhanced Trait", "Grip"):
        return "trait"
    elif any(key in pci for key in ORIGIN_PCI) or plug_def.get('itemTypeDisplayName') == "Origin Trait":
        return "origin_trait"
    elif 'masterworks' in pci and name.startswith('Masterworked:'):
        return "masterwork"
    elif "shader" in pci:
        return "shader"
    elif "weapon.mod_guns" in pci or "weapon mod" in item_type_display_name:
        return "weapon_mod"
    else:
        return "other"
    
# 1. Identify all trait sockets (by your PCI logic or however you already do it)
trait_socket_indexes = []
for socket_index, plug_defs in socket_plug_defs.items():
    # If any plug in this socket is a trait, consider this a trait socket
    if any(get_plug_category(plug_def) == "trait" for plug_def in plug_defs if plug_def):
        trait_socket_indexes.append(socket_index)

# 2. Sort trait sockets for consistent ordering
trait_socket_indexes = sorted(trait_socket_indexes)

# 3. Print with col3_trait1/col4_trait2 labels
for socket_index, plug_defs in socket_plug_defs.items():
    for plug_def in plug_defs:
        if not plug_def:
            continue
        name = plug_def['displayProperties']['name']
        category = get_plug_category(plug_def)
        item_hash = plug_def.get('hash')
        # Determine trait column label if this is a trait
        if category == "trait":
            if socket_index == trait_socket_indexes[0]:
                trait_label = "col3_trait1"
            elif len(trait_socket_indexes) > 1 and socket_index == trait_socket_indexes[1]:
                trait_label = "col4_trait2"
            else:
                trait_label = "trait"
            print(f"Socket {socket_index}: {name} (hash: {item_hash}) -> {trait_label}")
        else:
            print(f"Socket {socket_index}: {name} (hash: {item_hash}) -> {category}")

Socket 1: Hard Launch (hash: 981914802) -> col1_barrel
Socket 1: Quick Launch (hash: 3525010810) -> col1_barrel
Socket 2: Black Powder (hash: 1996142143) -> col2_magazine
Socket 2: Implosion Rounds (hash: 3492396210) -> col2_magazine
Socket 3: Field Prep (hash: 2869569095) -> col3_trait1
Socket 3: Impulse Amplifier (hash: 951095735) -> col3_trait1
Socket 4: Explosive Light (hash: 3194351027) -> col4_trait2
Socket 5: Default Shader (hash: 4248210736) -> shader
Socket 6: Adept Blast Radius (hash: 229003538) -> weapon_mod
Socket 6: Adept Projectile Speed (hash: 2299766748) -> weapon_mod
Socket 6: Adept Handling (hash: 4278960718) -> weapon_mod
Socket 6: Adept Reload (hash: 634781242) -> weapon_mod
Socket 6: Adept Counterbalance (hash: 1525622117) -> weapon_mod
Socket 6: Adept Targeting (hash: 1710791394) -> weapon_mod
Socket 8: Stunning Recovery (hash: 1281216113) -> origin_trait
Socket 8: Vanguard's Vindication (hash: 744217850) -> origin_trait
